In [5]:
import torch 
from torch import nn
from torch.nn import functional as F

net  = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X = torch.rand(2,20)
print(X.shape)
Y = net(X)
print(Y)

torch.Size([2, 20])
tensor([[ 0.2302,  0.1306,  0.1667, -0.1130, -0.0492,  0.0149,  0.1210,  0.1598,
          0.1717, -0.0902],
        [ 0.2489,  0.0963,  0.1219,  0.0034, -0.0978,  0.0262, -0.0386,  0.0338,
          0.1104, -0.0597]], grad_fn=<AddmmBackward>)


In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__() #这里是调用他的父类
        self.hidden = nn.Linear(20,256)#这里是定义这个类里面的层
        self.out = nn.Linear(256,10)#这里是定义输出层
        
    def forward(self,X):#定义前向传播的方法 #注意到这里的self，是python自己的写法，类方法约定俗成的
        return self.out(F.relu(self.hidden(X)))#返回值为最终的输出值


In [7]:
net =  MLP() #这里是用类生成一个实例net
net(X) #然后直接调用就可以了

tensor([[ 0.0905,  0.0464,  0.2100, -0.0021,  0.0086,  0.0489, -0.0803, -0.1451,
         -0.0888,  0.1673],
        [ 0.1483,  0.1591,  0.2297, -0.0131, -0.0791,  0.0283, -0.0912, -0.1453,
         -0.0006,  0.2272]], grad_fn=<AddmmBackward>)

自己写个Sequential

In [11]:
class Mysequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for block in args:
            self._modules[block] = block #这是pytorch自动识别到的方法，按顺序存放这些层,以字典的方法来存放
    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X
    


In [15]:
net  = Mysequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.0429,  0.2152,  0.1097,  0.1131,  0.1467,  0.1590, -0.0080,  0.0759,
         -0.0938, -0.1550],
        [ 0.0852,  0.1676,  0.0960,  0.0367,  0.0435,  0.0879,  0.0172, -0.0665,
         -0.1063, -0.0638]], grad_fn=<AddmmBackward>)

自己写个Module

In [17]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad = False)
        self.linear  = nn.Linear(20,20)
    def forward(self,X):
        X = self.linear(X) #先做线性
        X = F.relu(torch.mm(X,self.rand_weight)+1) #与系数做乘法，再用激活函数
        X = self.linear(X) #再做线性
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net =  FixedHiddenMLP()
net(X)   
            

tensor(0.0438, grad_fn=<SumBackward0>)

混合使用sequential来自定义Module

In [23]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net  =nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                 nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self,X):
        return self.linear(self.net(X))
chimera = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)




tensor(-0.0892, grad_fn=<SumBackward0>)